<a href="https://colab.research.google.com/github/MaidenTaief/Music_Final/blob/main/Merge_genre_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Dataset path
dataset_path = '/content/drive/My Drive/DATASET'

# Load Data
genre_csv = pd.read_csv(f'{dataset_path}/data_w_genres.csv')
label_data = pd.read_csv(f'{dataset_path}/data_final_with_descriptive_labels.csv')


import ast  # For safely evaluating strings containing Python expressions

def clean_artists_names(x):
    if x.startswith('['):
        x = ast.literal_eval(x)
        return ', '.join(x)
    return x.strip()

genre_csv['artists'] = genre_csv['artists'].str.strip()
label_data['artists'] = label_data['artists'].apply(clean_artists_names)

# Merge the datasets on the 'artists' column
merged_data = pd.merge(
    label_data[['artists', 'name', 'popularity', 'year', 'cluster', 'Cluster_Description']],
    genre_csv[['artists', 'genres']],
    on='artists',
    how='left'
)

# Normalize genre data
merged_data['genres'] = merged_data['genres'].apply(lambda x: 'Unknown' if pd.isna(x) or x == '[]' else x)

def normalize_genres(genre_list):
    if genre_list == 'Unknown':
        return 'Unknown'
    genre_list = ast.literal_eval(genre_list) if isinstance(genre_list, str) else genre_list
    return ' / '.join(sorted(set(genre_list)))

merged_data['genres'] = merged_data['genres'].apply(normalize_genres)

print(merged_data.sample(10))
print(merged_data.info())
print(merged_data.describe())

                               artists  \
44556                      Eartha Kitt   
162778          Brian Eno, David Byrne   
79098   Francisco Canaro, Mario Alonso   
48909                      Little Feat   
164994           El Haragán y Compañía   
104679                     Eva Cassidy   
3347                     Benny Goodman   
81283                  Louis Armstrong   
128780                    The Olympics   
65326                              Yes   

                                                     name  popularity  year  \
44556                          Uska Dara - A Turkish Tale          23  1956   
162778                                 Moonlight in Glory          21  1981   
79098                     Noche de Niebla - Remasterizado           0  1953   
48909   Spanish Moon - Live at Lisner Auditorium, Wash...          35  1978   
164994                                  Antes Me Gustabas          40  1992   
104679                                       Kathy's Song      

In [4]:
merged_data.to_csv(f'{dataset_path}/data_final_cleaned1.csv', index=False)